In [1]:
import numpy as np
import pandas as pd
from scipy.stats import gmean
from tqdm import tqdm
import gffutils
import os.path
#from liftover import ChainFile
from collections import defaultdict
from math import ceil
import pyfastx
import re
from scipy.sparse.csgraph import connected_components
from glob import glob
from scipy.sparse import lil_matrix,csr_matrix,coo_matrix,dok_matrix, save_npz
import pickle
#
blacklist = pd.read_csv('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX-V8/ENCFF220FIN.bed',header=None,sep='\t')

/home/benediktj/Python3.9.1/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
simpileRepeats = pd.read_csv('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX-V8/simpleRepeat.txt',header=None,sep='\t')

In [3]:
#gffutils.create_db('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX/gencode.v42.annotation.gtf', "/odinn/tmp/benediktj/Data/SplicePrediction-GTEX/gencode.v42.annotation.db", force=True,disable_infer_genes=True, disable_infer_transcripts=True)

In [4]:
#gtf_gencode = gffutils.FeatureDB('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX-V8/gencode.v42.annotation.db')

In [5]:
leafcutter_counts = glob('/nfs/odinn/data/dataprocessing/rnasplice-blood/curry/leafCutter/*counts.tab')
leafcutter_summary = glob('/nfs/odinn/data/dataprocessing/rnasplice-blood/curry/leafCutter/*summary.tab')

In [6]:
df1 = pd.read_csv('/odinn/users/gislih/RNA/requests/rna_paper/splice_anno/junction_db.tsv.gz', compression='gzip', sep='\t')

In [6]:
#junctions = pd.read_csv('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX-V8/GTEx_Analysis_2017-06-05_v8_STARv2.5.3a_junctions.gct', nrows=10, skiprows=2,sep='\t')

In [7]:
#junctions = junctions.sort_values('Chr')

In [8]:
def findLowCountJunctions(leafcutter_counts,blacklist,simpileRepeats):
    discardJunctionDict = defaultdict(bool)
    discardReason = defaultdict(bool)
    for f_i,file in enumerate(leafcutter_counts):
        junctions = pd.read_csv(file,sep='\t')
        chrom =leafcutter_counts[f_i].split('_')[1]
        print(chrom)
        tmp2 = junctions.iloc[:,1:]
        transcriptCount = np.sum(tmp2,axis=1)
        tmp2.columns = [x.split('-')[1] for x in junctions.columns[1:]]
        tmp2 = tmp2.T.groupby(tmp2.columns).sum().T

        includeJunction = np.sum(tmp2 > 0,axis=1)>=4

        blacklist_chrom = blacklist[blacklist[0]==chrom]
        simpileRepeats_chrom = simpileRepeats[simpileRepeats[1]==chrom]

        for i,junction in tqdm(enumerate(junctions.id.values)):
            chrom,start,end,cluster = junctions.iloc[i,0].split(':')
            start,end = int(start),int(end)
            cond1 = includeJunction[i]==False

            if cond1:
                discardJunctionDict[junction] = True
                discardReason[junction] = 'LowReadCount'
                #prev_chrom = chrom
                continue

            #if prev_chrom != chrom:
             #   blacklist_chrom = blacklist[blacklist[0]==chrom]
            tmp = blacklist_chrom[np.logical_not(np.any([blacklist_chrom[1]>=end, blacklist_chrom[2]<=start],0))]
            cond2 = False
            for i_b in range(tmp.shape[0]):
                if (np.isin(start, range(tmp.iloc[i_b,1],tmp.iloc[i_b,2]+1)) or np.isin(end, range(tmp.iloc[i_b,1],tmp.iloc[i_b,2]+1))):
                    cond2 = True
                    break

            if cond2:
                discardJunctionDict[junction] = True
                discardReason[junction] = 'InBlacklistedRegion'
                #prev_chrom = chrom
                continue

            #if prev_chrom != chrom:
            #    simpileRepeats_chrom = simpileRepeats[simpileRepeats[1]==chrom]

            tmp = simpileRepeats_chrom[np.logical_not(np.any([simpileRepeats_chrom[2]>=end,simpileRepeats_chrom[3]<=start],0))]
            cond3 = False
            for i_b in range(tmp.shape[0]):
                if (np.isin(start, range(tmp.iloc[i_b,2],tmp.iloc[i_b,3]+1)) or np.isin(end, range(tmp.iloc[i_b,2],tmp.iloc[i_b,3]+1))):
                    cond3 = True
                    break

            if cond3:
                discardJunctionDict[junction] = True
                discardReason[junction] = 'InRepeatRegion'
                continue
        
    return discardJunctionDict,discardReason

In [9]:
discardJunctionDict,discardReason = findLowCountJunctions(leafcutter_counts,blacklist,simpileRepeats)

chr10


10380it [00:14, 709.97it/s]


chr11


15511it [00:17, 901.36it/s] 


chr12


...

12764it [00:18, 698.28it/s] 


chr8


8570it [00:12, 694.76it/s] 


chr9


10551it [00:13, 791.41it/s] 


In [6]:
with open('/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood/discardJunctions.pkl', 'rb') as f:
    discardJunctionDict = pickle.load(f)

with open('/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood/discardReason.pkl', 'rb') as f:
     discardJunctionDict = pickle.load(f)


In [7]:
#with open('/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood/discardJunctions.pkl', 'wb') as f:
#    pickle.dump(discardJunctionDict, f)

In [8]:
#with open('/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood/discardReason.pkl', 'wb') as f:
#    pickle.dump(discardReason, f)

In [7]:
#leafcutterFiles = glob('/odinn/tmp/bjarnih/RNA/leafCutter/GTEx/*/meta_results/clusters_*_summary.tab')
leafcutterFiles = leafcutter_summary
chrmToLeafcutterFiles = defaultdict(list)
for file in leafcutterFiles:
    chrm = file.split('/')[-1].split('_')[1]
    chrmToLeafcutterFiles[chrm].append(file)

In [8]:
def getCombinedLeafCutterDF(chrm,chrmToLeafcutterFiles):
    files = chrmToLeafcutterFiles[chrm]
    for i in range(len(files)):
        if i == 0:
            df = pd.read_csv(files[i],sep='\t')
        else:
            df = pd.concat([df,pd.read_csv(files[i],sep='\t')],axis=0)
            df = df.drop_duplicates('splice_event_id')
    return df.sort_values('Start')

In [9]:
df1 = getCombinedLeafCutterDF('chr1',chrmToLeafcutterFiles)

In [19]:
df1

,Chrom,Start,End,ClusterID,splice_event_id,Gene_name,Gene_id,Strand,Gene_start_min,Gene_end_max,...,cluster_type,median_psi,max_psi,prop_NA,prop_zero,num_unique,low_variability,any_annotated_in_cluster,any_gene_expressed,protein_coding
0,chr1,182746,183015,0,chr1:182746:183015:0,FO538757.2,ENSG00000279928,+,182393,184158,...,alt_acceptor,0.000000,1.000000,0.467503,0.730008,271,False,True,True,True
1,chr1,182746,183113,0,chr1:182746:183113:0,FO538757.2,ENSG00000279928,+,182393,184158,...,alt_acceptor,0.166667,1.000000,0.467503,0.191604,521,False,True,True,True
2,chr1,182746,183131,0,chr1:182746:183131:0,FO538757.2,ENSG00000279928,+,182393,184158,...,alt_acceptor,0.800000,1.000000,0.467503,0.000631,536,False,True,True,True
3,chr1,182746,183164,0,chr1:182746:183164:0,FO538757.2,ENSG00000279928,+,182393,184158,...,alt_acceptor,0.000000,0.714286,0.467503,0.825021,231,False,True,True,True
4,chr1,749381,752447,1,chr1:749381:752447:1,"RP11-206L10.2,RP5-857K21.4","ENSG00000228327,ENSG00000230021",-,585989,827796,...,alt_donor,0.000000,1.000000,0.816450,0.578144,75,False,False,True,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25872,chr1,248906342,248913815,4935,chr1:248906342:248913815:4935,PGBD2,ENSG00000185220,+,248906196,248919946,...,alt_donor,0.971429,1.000000,0.010982,0.000000,600,False,True,True,True
25874,chr1,248913879,248916601,4936,chr1:248913879:248916601:4936,PGBD2,ENSG00000185220,+,248906196,248919946,...,complex,0.474788,1.000000,0.020731,0.006923,1257,False,True,True,True
25875,chr1,248913879,248917278,4936,chr1:248913879:248917278:4936,PGBD2,ENSG00000185220,+,248906196,248919946,...,complex,0.428571,1.000000,0.020731,0.002575,1262,False,True,True,True
25873,chr1,248913879,248914438,4936,chr1:248913879:248914438:4936,PGBD2,ENSG00000185220,+,248906196,248919946,...,complex,0.040000,0.750000,0.020731,0.349754,573,False,True,True,True


In [11]:
df2 = pd.read_csv('/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood-050623/filtered_junctions.gor',sep='\t')

In [23]:
df2[df2['filtered__Gene_name']=='SAMD11']

,Chrom,Startx,End,num_Individs,sum_Fcount_passed,sum_Max_min_overhang_passed,sum_Left_entropy_passed,sum_Right_entropy_passed,max_num_Tests_passed,sum_F_OH,sum_F_OH_LE,sum_F_OH_LE_RE,filtered__End,filtered__Strand,filtered__Gene_name,filtered__max_Fcount,filtered__med_Prop_multy_mapped,filtered__expr_cutoff,filtered__max_Fcount_passed,filtered__med_Prop_multy_mapped_passed
47,chr1,943058,943252,37,2,31,1,1,4,2,1,1,943252,+,SAMD11,4,0.0,0.011258,1,1
48,chr1,943808,943907,139,3,56,1,1,4,3,1,1,943907,+,SAMD11,6,0.0,0.011258,1,1


In [31]:
def getCombinedLeafCutterDF(chrm,chrmToLeafcutterFiles):
    files = chrmToLeafcutterFiles[chrm]
    for i in range(len(files)):
        if i == 0:
            df = pd.read_csv(files[i],sep='\t')
        else:
            df = pd.concat([df,pd.read_csv(files[i],sep='\t')],axis=0)
            df = df.drop_duplicates('splice_event_id')
    #df = df[['Chrom','Start','End','Gene_id','ClusterID']]
    #df = pd.concat([rna_splice_junctions[rna_splice_junctions['Chrom']==chrm],df],axis=0)
    return df.sort_values('Start')

leafcutterFiles = glob('/odinn/tmp/bjarnih/RNA/leafCutter/GTEx/*/meta_results/clusters_*_summary.tab')
chrmToLeafcutterFiles = defaultdict(list)
for file in leafcutterFiles:
    chrm = file.split('/')[-1].split('_')[1]
    chrmToLeafcutterFiles[chrm].append(file)

In [32]:
df3 = getCombinedLeafCutterDF('chr1',chrmToLeafcutterFiles)

In [49]:
intron_junctions = getJunctions(gtf_gencode,gtf_gencode['ENST00000616016.5'],'-')

In [50]:
intron_junctions

array([[944574, 943698],
       [943808, 943253],
       [943377, 942559],
       [943058, 942410],
       [942488, 942136],
       [942251, 941144],
       [941306, 939275],
       [939412, 939040],
       [939129, 935772],
       [935896, 931039],
       [931089, 930155],
       [930336, 925922],
       [926013, 923923]])

In [39]:
df3[df3['Start']==943808].sort_values('End')

,Chrom,Start,End,ClusterID,splice_event_id,Gene_name,Gene_id,Strand,Gene_start_min,Gene_end_max,...,is_annotated,cluster_type,median_psi,max_psi,prop_NA,prop_zero,num_unique,low_variability,any_annotated_in_cluster,any_gene_expressed
241,chr1,943808,943893,clu_45656,chr1:943808:943893:clu_45656,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,False,alt_acceptor,0.000000,0.142857,0.002688,0.598383,97,False,True,True
257,chr1,943808,943893,clu_48001,chr1:943808:943893:clu_48001,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,False,alt_acceptor,0.000000,0.400000,0.038229,0.884937,38,False,True,True
234,chr1,943808,943893,clu_49105,chr1:943808:943893:clu_49105,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,False,alt_acceptor,0.017938,0.063158,0.004219,0.156780,173,False,True,True
225,chr1,943808,943893,clu_45793,chr1:943808:943893:clu_45793,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,False,alt_acceptor,0.000000,0.300000,0.010336,0.757180,64,False,True,True
90,chr1,943808,943893,clu_34660,chr1:943808:943893:clu_34660,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,False,alt_acceptor,0.000000,0.500000,0.007937,0.800000,24,False,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216,chr1,943808,943908,clu_45431,chr1:943808:943908:clu_45431,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,True,alt_acceptor,1.000000,1.000000,0.000000,0.000000,75,False,True,True
249,chr1,943808,943908,clu_53580,chr1:943808:943908:clu_53580,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,True,alt_acceptor,1.000000,1.000000,0.007519,0.001894,64,False,True,True
226,chr1,943808,943908,clu_45793,chr1:943808:943908:clu_45793,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,True,alt_acceptor,1.000000,1.000000,0.010336,0.000000,64,False,True,True
158,chr1,943808,943908,clu_41290,chr1:943808:943908:clu_41290,SAMD11,ENSG00000187634,+,923928.0,944581.0,...,True,alt_acceptor,1.000000,1.000000,0.053892,0.006329,29,False,True,True


In [37]:
df2[df2['filtered__Gene_name']=='SAMD11']

,Chrom,Startx,End,num_Individs,sum_Fcount_passed,sum_Max_min_overhang_passed,sum_Left_entropy_passed,sum_Right_entropy_passed,max_num_Tests_passed,sum_F_OH,sum_F_OH_LE,sum_F_OH_LE_RE,filtered__End,filtered__Strand,filtered__Gene_name,filtered__max_Fcount,filtered__med_Prop_multy_mapped,filtered__expr_cutoff,filtered__max_Fcount_passed,filtered__med_Prop_multy_mapped_passed
47,chr1,943058,943252,37,2,31,1,1,4,2,1,1,943252,+,SAMD11,4,0.0,0.011258,1,1
48,chr1,943808,943907,139,3,56,1,1,4,3,1,1,943907,+,SAMD11,6,0.0,0.011258,1,1


In [ ]:
intron_junctions = getJunctions(gtf_gencode,gtf_gencode['ENST00000327044.7'],'-')

In [48]:
intron_junctions

array([[944800, 945057],
       [945146, 945518],
       [945653, 946173],
       [946286, 946402],
       [946545, 948131],
       [948232, 948490],
       [948603, 951127],
       [951238, 952000],
       [952139, 952412],
       [952600, 953175],
       [953288, 953782],
       [953892, 954004],
       [954082, 955923],
       [956013, 956095],
       [956215, 956894],
       [957025, 957099],
       [957273, 958929],
       [959081, 959215]])

In [53]:
df3[df3['Start']==944800]['End'].unique()

array([945057, 945042])

In [22]:
df2[df2['filtered__Gene_name']=='NOC2L']

,Chrom,Startx,End,num_Individs,sum_Fcount_passed,sum_Max_min_overhang_passed,sum_Left_entropy_passed,sum_Right_entropy_passed,max_num_Tests_passed,sum_F_OH,sum_F_OH_LE,sum_F_OH_LE_RE,filtered__End,filtered__Strand,filtered__Gene_name,filtered__max_Fcount,filtered__med_Prop_multy_mapped,filtered__expr_cutoff,filtered__max_Fcount_passed,filtered__med_Prop_multy_mapped_passed
49,chr1,944800,945041,17831,17564,17823,17257,17290,4,17564,17257,17181,945041,-,NOC2L,457,0.0,1.846592,1,1
50,chr1,944800,945056,17844,17807,17844,17707,17727,4,17807,17707,17693,945056,-,NOC2L,762,0.0,1.846592,1,1
51,chr1,944819,945041,1791,48,1642,2,2,4,45,2,2,945041,-,NOC2L,8,0.0,1.846592,1,1
52,chr1,944819,945056,3471,114,3336,13,16,4,113,13,11,945056,-,NOC2L,13,0.0,1.846592,1,1
53,chr1,945146,945517,17848,17839,17847,17822,17821,4,17839,17822,17817,945517,-,NOC2L,1497,0.0,1.846592,1,1
54,chr1,945422,945517,10114,1481,3380,41,148,4,736,41,31,945517,-,NOC2L,30,0.0,1.846592,1,1
55,chr1,945653,946146,16402,9686,16067,7628,7449,4,9678,7628,7109,946146,-,NOC2L,89,0.0,1.846592,1,1
56,chr1,945653,946172,17847,17841,17846,17810,17806,4,17841,17810,17800,946172,-,NOC2L,1349,0.0,1.846592,1,1
57,chr1,945653,946401,665,11,554,4,4,4,9,4,4,946401,-,NOC2L,40,0.0,1.846592,1,1
58,chr1,946286,946401,17845,17813,17845,17734,17737,4,17813,17734,17718,946401,-,NOC2L,926,0.0,1.846592,1,1


In [43]:
#fasta = pyfastx.Fasta('/odinn/tmp/benediktj/Data/SplicePrediction-050422/genome.fa')
gtf_gencode = gffutils.FeatureDB('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX-V8/gencode.v42.annotation.db')
fasta = pyfastx.Fasta('/odinn/tmp/benediktj/Data/SplicePrediction-GTEX-V8/GRCh38.p13.genome.fa')

In [43]:
data_dir = '/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood'

seqData = {}

CHROM_GROUP = ['chr1', 'chr3', 'chr5', 'chr7', 'chr9',
'chr11', 'chr13', 'chr15', 'chr17', 'chr19', 'chr21',
'chr2', 'chr4', 'chr6', 'chr8', 'chr10', 'chr12',
'chr14', 'chr16', 'chr18', 'chr20', 'chr22', 'chrX', 'chrY']

for chrom in CHROM_GROUP:
        seqData[chrom] = dok_matrix((len(fasta[chrom]), 5), dtype=np.int8)

seqData[chrom] = dok_matrix((len(fasta[chrom]), 5), dtype=np.int8)

In [41]:
def create_datapoints(seq, strand, tx_start, tx_end):
    # This function first converts the sequence into an integer array, where
    # A, C, G, T, Missing are mapped to 1, 2, 3, 4, 5 respectively. If the strand is
    # negative, then reverse complementing is done. . It then calls reformat_data and one_hot_encode

    seq = seq.upper()
    seq = re.sub(r'[^AGTC]', '5',seq)
    seq = seq.replace('A', '1').replace('C', '2')
    seq = seq.replace('G', '3').replace('T', '4')

    tx_start = int(tx_start)
    tx_end = int(tx_end) 

    Y_idx = []
    
    X0 = np.asarray([int(x) for x in seq])

    X = one_hot_encode(X0)

    return X

def ceil_div(x, y):
    return int(ceil(float(x)/y))


IN_MAP = np.asarray([[0, 0, 0, 0,0],
                     [1, 0, 0, 0,0],
                     [0, 1, 0, 0,0],
                     [0, 0, 1, 0,0],
                     [0, 0, 0, 1,0],
                    [0, 0, 0, 0,1]])
# One-hot encoding of the inputs: 0 is for padding, and 1, 2, 3, 4 correspond
# to A, C, G, T, Missing respectively.

OUT_MAP = np.asarray([[1, 0, 0],
                      [0, 1, 0],
                      [0, 0, 1],
                      [0, 0, 0]])

def one_hot_encode(Xd):
    return IN_MAP[Xd.astype('int8')]

def getJunctions(gtf,transcript,strand):
    #transcript = gtf[transcript_id.split('.')[0]]
    exon_junctions = []
    tx_start = int(transcript[3])
    tx_end = int(transcript[4])
    exons = gtf.children(transcript, featuretype="exon")
    for exon in exons:
        exon_start = int(exon[3])
        exon_end = int(exon[4])
        exon_junctions.append((exon_start,exon_end))

    intron_junctions = []

    if strand=='+':
        intron_start = exon_junctions[0][1]
        for i,exon_junction in enumerate(exon_junctions[1:]):
            intron_end = exon_junction[0]
            intron_junctions.append((intron_start,intron_end))
            if i+1 != len(exon_junctions[1:]):
                intron_start = exon_junction[1]

    elif strand=='-':
        exon_junctions.reverse()
        intron_start = exon_junctions[0][1]
        for i,exon_junction in enumerate(exon_junctions[1:]):
            intron_end = exon_junction[0]
            intron_junctions.append((intron_start,intron_end))
            if i+1 != len(exon_junctions[1:]):
                intron_start = exon_junction[1]

    return np.array(intron_junctions)

In [45]:
!rm /odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood/annotation_rnasplice-blood.txt

rm: cannot remove ‘/odinn/tmp/benediktj/Data/SplicePrediction-rnasplice-blood/annotation_rnasplice-blood.txt’: No such file or directory


In [46]:
transcripts = gtf_gencode.features_of_type('transcript')
gene_to_label = {}
save_seq = True
prev_chrom = 'chr1'
leaf_cutter_junctions = getCombinedLeafCutterDF(prev_chrom,chrmToLeafcutterFiles)
gene_ids_containing_junction = leaf_cutter_junctions['Gene_id'].apply(lambda x: x.split(',')).values
for transcript in tqdm(transcripts): 
    chrom,gene_start,gene_end,strand,gene_id,transcript_id,gene_type,gene_name,level = transcript[0],transcript[3],transcript[4],transcript[6],transcript[8]['gene_id'][0],transcript[8]['transcript_id'][0],transcript[8]['gene_type'][0],transcript[8]['gene_name'][0],transcript[8]['level'][0]
    
    try:
        cond1 = 'Ensembl_canonical' in transcript[8]['tag']
    except:
        continue
    cond2 = gene_type=='protein_coding'
    cond3 = int(level)<3
    if cond1 and cond2 and cond3:
        intron_junctions = getJunctions(gtf_gencode,transcript,strand)
        junction_starts = defaultdict(int)
        junction_ends = defaultdict(int)

        if len(intron_junctions>0):
            if chrom!=prev_chrom:
                leaf_cutter_junctions = getCombinedLeafCutterDF(chrom,chrmToLeafcutterFiles)
                gene_ids_containing_junction = leaf_cutter_junctions['Gene_id'].apply(lambda x: x.split(',')).values
            for junction in intron_junctions:
                junction_starts[junction[0]] = 1
                junction_ends[junction[1]] = 1
            simple_gene_id = gene_id.split('.')[0]

            alt_junctions = leaf_cutter_junctions[[(simple_gene_id in x) for x in gene_ids_containing_junction]]

            if alt_junctions.shape[0]>0:
                clusters = defaultdict(int)
                for i,pos in enumerate(alt_junctions['Start']):
                    if junction_starts[pos] == 1:
                        clusters[alt_junctions.iloc[i,:]['ClusterID']] = 1
                for i,pos in enumerate(alt_junctions['End']):
                    if junction_ends[pos] == 1:
                        clusters[alt_junctions.iloc[i,:]['ClusterID']] = 1
                for cluster in clusters.keys():
                    cluster_junctions = alt_junctions[alt_junctions['ClusterID']==cluster][['Start','End']]
                    for i_junc in range(cluster_junctions.shape[0]):
                        start,end = cluster_junctions.iloc[i_junc,:]['Start'],cluster_junctions.iloc[i_junc,:]['End']
                        junction_id = '{}:{}:{}:{}'.format(chrom,start,end,cluster)
                        if not discardJunctionDict[junction_id]:
                            junction_starts[start] = 1
                            junction_ends[end] = 1

                junction_starts = {k:v for k,v in junction_starts.items() if v != 0}
                junction_ends = {k:v for k,v in junction_ends.items() if v != 0}
                gene_to_label[gene_id] = [junction_starts, junction_ends]

                if save_seq:
                    seq = fasta[chrom][int(gene_start)-1:int(gene_end)]
                    seq = seq.seq
                    X = create_datapoints(seq, strand, gene_start, gene_end)
                    seqData[chrom][int(gene_start)-1:int(gene_end)] = X
                    jn_start = list(junction_starts.keys())
                    jn_end = list(junction_ends.keys())
                    name = '{}\t{}\t{}\t{}'.format(gene_name,gene_id,transcript_id,level)

                    if strand=='+':
                        with open('{}/annotation_rnasplice-blood.txt'.format(data_dir), 'a') as the_file:
                            the_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(name,chrom,strand,gene_start,gene_end,','.join([str(x) for x in jn_start]),','.join([str(x) for x in jn_end])))
                    if strand=='-':
                        with open('{}/annotation_rnasplice-blood.txt'.format(data_dir,), 'a') as the_file:
                            the_file.write('{}\t{}\t{}\t{}\t{}\t{}\t{}\n'.format(name,chrom,strand,gene_start,gene_end,','.join([str(x) for x in jn_end]),','.join([str(x) for x in jn_start])))

                    if chrom!=prev_chrom:
                        save_npz('{}/sparse_sequence_data/{}.npz'.format(data_dir,prev_chrom), seqData[prev_chrom].tocoo())
                        del seqData[prev_chrom]

                    prev_chrom = chrom
                
    #except:
     #   pass
        #print(gene[2])
        #print(gene[8]['transcript_support_level'])

243275it [3:02:00, 22.28it/s] 


UnboundLocalError: local variable 'df' referenced before assignment

In [50]:
save_npz('{}/sparse_sequence_data/{}.npz'.format(data_dir,prev_chrom), seqData[prev_chrom].tocoo())
#del seqData[prev_chrom]

In [51]:
with open('{}/gene_to_label.pickle'.format(data_dir), 'wb') as handle:
    pickle.dump(gene_to_label, handle, protocol=pickle.HIGHEST_PROTOCOL)